In [ ]:
import os
import sys
import traci
import sumolib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
from xml.dom import minidom
from sumolib import checkBinary
import math
from scipy.stats import norm
import ray
import pickle

In [ ]:
# check env
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    raise EnvironmentError("Please declare environment variable 'SUMO_HOME'")

In [ ]:
# model folder for driving behavior
model_folder="model_params/"

# read driving behavior models

PT_param_data={0: pd.read_csv(model_folder+"merged_PT_S.csv"),
                1: pd.read_csv(model_folder+"merged_PT_A.csv"),
                   2: pd.read_csv(model_folder+"merged_PT_L.csv")
                  }
# IDM by vehicle type
IDM_param_data={0: pd.read_csv(model_folder+"merged_IDM_S.csv"),
                   1: pd.read_csv(model_folder+"merged_IDM_A.csv"),
                   2: pd.read_csv(model_folder+"merged_IDM_L.csv")}

In [ ]:
# initialization function for all dataset
# Note: the pickle files for next lanes are needed because Sumo cannot auytomatically detect the downstream lanes for customized models

def initialize():
    od_demand={} # the demands are stored as dictionaries of dictionary
    if site=="I90_94":
        # the names of input and output edges
        in_names={1:"main_in", 2:"on1"}
        out_names={1:"off1",2:"off2", 3:"off3", 4:"off4", 5:"main_I90_out", 6:"main_I94_out" }
        downstream_outs={1:{1,2,3,4,5,6}, 2:{2,3,4,5,6}} # make sure that the exist is downstream of the entrance
        # define edges as in sumo (can be changes in the future for an easier name)
        in_edges={1:"124471132", 2:"24521704#3.24"}
        out_edges={1:"24306156",2:"24521700#0", 3:"27878350", 4:"27862401", 5:"380664935", 6:"907236679" }
        # user defined demand
        # edges where MLC are made, and the directions
        in_MLCs={"124471132":{}, "24521704#3.24":{"121980283":1}}
        out_MLCs={"24306156":{}, "24521700#0":{}, "27878350":{}, "27862401":{}, "380664935":{}, "907236679":{}}
        with open('configs/next_lanes_I90_94.pickle', 'rb') as handle:
            next_lanes_dict = pickle.load(handle)
        
    elif site=="I294":
        in_names={1:"main_in", 2:"on1", 3:"on2", 4:"on3"}
        out_names={1:"off1", 2:"off2", 3:"main_out"}
        in_edges={1:"992387151#1", 2:"38878546.568", 3:"31144126.151", 4:"24067152.262"}
        out_edges={1:"24067154", 2:"24067155", 3:"377623249#1-AddedOnRampEdge"}
        downstream_outs={1:{1,2,3}, 2:{1,2,3}, 3:{3}, 4:{3} }
        
        in_MLCs={"61857431#1":{}, "61857388":{"121980283":1}, "24067978":{}, "38878546.568":{}, "31144126.151":{}, "24067152.262":{}}
        out_MLCs={"33726629":{},"24067154":{}, "24067155":{}, "377623249#1":{}}
        
        with open('configs/next_lanes_I294.pickle', 'rb') as handle:
            next_lanes_dict = pickle.load(handle)
    
    elif site=="I395":
        in_names={1: "main_in", 2:"on1"}
        out_names={1: "off1", 2:"main_out"}
        in_edges={1:"6063187-AddedOffRampEdge", 2:"297204299"}
        out_edges={1:"397355152", 2:"121980283.9"}
        downstream_outs={1:{1,2}, 2:{2} }
        
        in_MLCs={"6063187-AddedOffRampEdge":{}, "297204299":{"121980283":-1}} # for each origin what 
        out_MLCs={"397355152":{"6063187-AddedOffRampEdge":-1}, "121980283.9":{}}
        
        with open('configs/next_lanes_I395.pickle', 'rb') as handle:
            next_lanes_dict = pickle.load(handle)

    # Defines the dlow rate for od demand
    # here we are assuming the flow between on-ramps and off-ramps are the same
    # Also the same flow for 
    # 
    for in_idx in in_names:
        od_demand[in_idx]={}
        in_name=in_names[in_idx]
        for out_idx in out_names:
            out_name=out_names[out_idx]
            if not out_idx in downstream_outs[in_idx]:
                od_demand[in_idx][out_idx]=0 #if the offramp is upstream of the on ramp
            elif ("main" in in_name) and ("main" in out_name):
                od_demand[in_idx][out_idx]=flow_main
            elif (("main" in in_name) and ( not "main" in out_name)):
                od_demand[in_idx][out_idx]=flow_main_to_ramp
            elif ((not "main" in in_name) and ("main" in out_name)):
                od_demand[in_idx][out_idx]=flow_ramp_to_main
            else:
                od_demand[in_idx][out_idx]=flow_ramp_to_ramp
            print("From " + in_name+" to "+out_name+": ", od_demand[in_idx][out_idx], "veh/hr" )


    all_edges=[]
    # Load the XML file into a format readable by sumo
    parsed_data = ET.parse('network_structure/'+site+'.net.xml')
    root = parsed_data.getroot()
    
    print(f"Root tag: {root.tag}, Root attributes: {root.attrib}")
    
    # Iterate through elements to get the set of edges
    for edge in root.findall(".//edge"):
        all_edges.append(edge.attrib['id'])
        
    return od_demand, all_edges, in_edges, out_edges, 0, 0, next_lanes_dict

def generate_vehicles(trial=None):
    # first define dictionaries for generation times and all od pairs
    
    all_generation_times={}
    origins=[]
    destinations=[]
    gen_times=[]
    
    for in_idx in od_demand:
        in_edge=in_edges[in_idx]
        all_generation_times[in_edge]={}
        for out_idx in od_demand[in_idx]:
            out_edge=out_edges[out_idx]
            demand_rate=od_demand[in_idx][out_idx]
            if demand_rate>0:
                # if non-zero demand, then the arrival of vehicles is assumed to be poisson
                generation_times=np.cumsum(np.maximum(np.random.exponential(3600/demand_rate, 100000), min_hw))
                generation_times=generation_times[(generation_times>=min_t) & (generation_times<=max_t)]
                
                for ll in range(len(generation_times)):
                    origins.append(in_edge)
                    destinations.append(out_edge)
                    gen_times.append(generation_times[ll])
    
    origins=np.array(origins)
    destinations=np.array(destinations)
    gen_times=np.array(gen_times)
    sort_indices=np.argsort(gen_times)

    # sorting is needed because SUMO assumes the trips' departure time is sorted
    gen_times=gen_times[sort_indices]
    origins=origins[sort_indices]
    destinations=destinations[sort_indices]
    
    
    # save the driving behavior parameters of vehicles into a dictionary, since they are not default in sumo
    # it is also faster to access elements in dicts than in sumo
    vehicle_ids=np.array([idx+1 for idx in range(len(gen_times))])

    PTs_by_id={}
    IDMs_by_id={}
    types_by_id={}
    lens_by_id={}
    
    trips_data=[]
    
    for i in range(len(vehicle_ids)):#trip in trips_data:
        id=str(vehicle_ids[i])
        
        veh_type=np.random.choice([0,1,2], p=[sv_rate, av_rate, lv_rate])
        types_by_id[id]=veh_type
        lens_by_id[id]=(veh_type==2)*12+(veh_type!=2)*4

        
    
        PT_cf_data=PT_param_data[int(veh_type)]
        sample_PT=PT_cf_data.sample()[['Tmax', 'Alpha', 'Beta', 'Wc', 'Gamma1','Gamma2', 'Wm']].values[0]
        PTs_by_id[id]=sample_PT
    
        IDM_cf_data=IDM_param_data[int(veh_type)]
        sample_IDM=IDM_cf_data.sample()[['T', 'a', 'b', 'v0', 'so', "delta"]].values[0]
        IDMs_by_id[id]=sample_IDM

        veh={"id": id, "depart": str(round(gen_times[i],1)), "from": origins[i] , "to": destinations[i], "type": veh_type, "departSpeed":sample_IDM[3] }
        trips_data.append(veh)
    

    root = ET.Element("trips")

    # Add trips to the root, which is readable by sumo
    for trip in trips_data:
        trip_element = ET.SubElement(root, "trip")
        trip_element.set("id", str(trip["id"]))
        trip_element.set("depart", trip["depart"])
        trip_element.set("from", trip["from"])
        trip_element.set("to", trip["to"])

        trip_element.set("departLane", "random") # random departure lane
         
        trip_element.set("departSpeed", str(trip["departSpeed"]))
        
    
    # convert the format and save it as a trip file
    
    tree = ET.ElementTree(root)
    xml_str = minidom.parseString(ET.tostring(root, 'utf-8')).toprettyxml(indent="  ")
    
    # Save the XML to a file
    output_file = "configs/"+ site +"/"+site+"_trial"+str(trial)+".trips.xml"
    with open(output_file, "w") as file:
        file.write(xml_str)
    
    print(f"Trips saved to {output_file}")

    return PTs_by_id,  IDMs_by_id, types_by_id, lens_by_id, trips_data
    


In [ ]:
# prospect theory model
def PT(veh_id, leader_id):
    
    #PTs_by_id 
    Tmax, Alpha, Beta, Wc, Gamma1,Gamma2, Wm = PTs_by_id[veh_id] # read the PT parameters for the current vehicle
    # delta_v is speed difference
    # gap is the leader location minus the current location minus leader length
    # speed is the current speed of the central vehicle
    
    # central vehicle
    x_a = traci.vehicle.getLanePosition(veh_id)
    position=traci.vehicle.getPosition(veh_id)
    v_a=traci.vehicle.getSpeed(veh_id)
    road= traci.vehicle.getRoadID(veh_id)
    lane= traci.vehicle.getLaneIndex(veh_id)

    if leader_id!=None:
        leader_len=lens_by_id[leader_id]
        v_lead=traci.vehicle.getSpeed(leader_id)
        x_lead=traci.vehicle.getLanePosition(leader_id)
    else: # virtual vehicle needed, to be verified
        leader_len=0
        v_lead=v_a+0.01
        x_lead=10000000# make it huge

    veh_gap=x_lead-x_a-leader_len # gap between leader and the current vehicle
    veh_delta_v= v_a-v_lead # speed difference  

    So_D = 3 #default value by Talebpour
    if (veh_gap - So_D) > 0.1:
        Seff = veh_gap - So_D
    else:
        Seff = 0.1 #default value by Talebpour

    if veh_delta_v > (Seff / Tmax):
        Tau = Seff / veh_delta_v
    else:
        Tau = Tmax

    if veh_delta_v == 0:
        veh_delta_v = 0.0000001 #default value by Talebpour
    if Alpha == 0:
        Alpha = 0.0000001 #default value by Talebpour
    Zprime = Tau / (2.0 * Alpha * v_a)
    Zdoubleprime = 0.0

    #if Wc * Zprime >= 1:
    if Wc * Zprime > 0:
        if (2.0 * math.log(Wc * Zprime)) >= 0:
            a0 = 1
            #Zstar = (-1 * math.sqrt(2.0 * math.log(Wc * Zprime))) / (math.sqrt(2.0 * math.pi)) #default by Talebpour
            Zstar = -math.sqrt(2.0 * math.log((a0 * Wc * Tau) / (2.0 * math.sqrt(2.0 * math.pi) * Alpha * v_a))) #changed to be consistent with paper
            if np.abs(Zstar) > 0.05:
                Zstar = 0.05 #added threshold to reduce fluctuations
    else:
        Zstar = 0.0
    Astar = (2.0 / Tau) * ((Seff / Tau) - veh_delta_v + (Alpha * v_a * Zstar))
    for NewtonCounter in range(3):
        X = Astar
        if X >= 0:
            if X == 0:
                X = 0.0000001 #default value by Talebpour
            Uptprime = Gamma1 * math.pow(X, Gamma1 - 1)
            Uptdoubleprime = Gamma1 * (Gamma1 - 1) * math.pow(X, Gamma1 - 2)
        else:
            Uptprime = Wm * Gamma2 * pow(-X, Gamma2 - 1)
            Uptdoubleprime = -Wm * Gamma2 * (Gamma2 - 1) * pow(-X, Gamma2 - 2)

        Z = (veh_delta_v + (0.5 * Astar * Tau) - (Seff / Tau)) / (Alpha * veh_delta_v)
        fn = norm.cdf(Z)

        F = Uptprime - Wc * fn * Zprime
        Fprime = Uptdoubleprime - Wc * fn * (Z * math.pow(Zprime, 2.0) + Zdoubleprime)
        if Fprime == 0:
            Fprime = 0.000000000001 #default value by Talebpour

        Astar = Astar - (F / Fprime)

    X = Astar
    if X >= 0:
        Uptprime = Gamma1 * math.pow(X, Gamma1 - 1)
        Uptdoubleprime = Gamma1 * (Gamma1 - 1) * math.pow(X, Gamma1 - 2)
    else:
        Uptprime = Wm * Gamma2 * pow(-X, Gamma2 - 1)
        Uptdoubleprime = -Wm * Gamma2 * (Gamma2 - 1) * pow(-X, Gamma2 - 2)
    Z = (veh_delta_v + (0.5 * Astar * Tau) - (Seff / Tau)) / (Alpha * veh_delta_v)
    fn = norm.cdf(Z)
    F = Uptprime - Wc * fn * Zprime
    Fprime = Uptdoubleprime - Wc * fn * (Z * math.pow(Zprime, 2.0) + Zdoubleprime)
    if Fprime == 0:
        Fprime = 0.000000000001

    Var = -1.0 / (Beta * Fprime)

    Random_Wiener = np.random.rand()
    Yt = math.exp(-1 * 0.1 / Tau) + math.sqrt(24.0 * 0.1 / Tau) * Random_Wiener #default value by Talebpour
    accl_cf = Astar + Var * Yt
    accl_ff = accl_max * (1 - (v_a / v_desired))

    accl_ = np.minimum(accl_cf, accl_ff)

    if accl_ > 3: #default value by Talebpour
        accl_ = 3
    elif accl_ < -8: #default value by Talebpour
        accl_ = -8

    return accl_  # return acceleration

In [ ]:

def IDM(veh_id, leader_id, leader_lane, adjust_length, x_a, v_a, lane, x_lead):
    # input:
    ## vehicle id, leader id, the lane segment of the leader (may not be exactly the same as the follower, since SUMo segments the lanes into segments)
    ## adjust_length is used to adjust the distance if the lane of the leader and follower are segmented into different lane ids, since sumo returns the position within the same lane index"
    ### it is the remaining distance in the follower's lane index added with the lengths of the links skipped
    ## x_a v_a is the position of the follower within its lane segment and its speed
    ## lane is the lane index of the follower
    # x_lead is the leader position within its lane
    idm_params=IDMs_by_id[veh_id]
    T, a, b, v0, s0, delta = idm_params # include error term as well potentially
    
    if leader_id==None:
        acc=a*(1-(v_a/v0)**delta)
    elif lane==leader_lane: # same lane segment
        leader_len=lens_by_id[leader_id]
        veh_len=lens_by_id[veh_id]
        v_lead=traci.vehicle.getSpeed(leader_id)
        x_lead=traci.vehicle.getLanePosition(leader_id)
        s_star=s0+v_a*T + v_a*(v_a-v_lead)/(2*np.sqrt(a*b))
        acc=a*(1-(v_a/v0)**delta-(s_star/(x_lead-x_a-0.5*(leader_len+veh_len)))**2)
    else: # different lane, use absolute distance
        leader_len=lens_by_id[leader_id]
        veh_len=lens_by_id[veh_id]
        v_lead=traci.vehicle.getSpeed(leader_id)
        s_star=s0+v_a*T + v_a*(v_a-v_lead)/(2*np.sqrt(a*b))
        acc=a*(1-(v_a/v0)**delta-(s_star/(x_lead+adjust_length-0.5*(leader_len+veh_len)))**2)

    
    return acc

In [ ]:
# we are not visualizing

#@ray.remote # need this if it is run remotely. Fine otherwise
def run_simulation(inputs):
    trial=inputs[0]
    PTs_by_id=inputs[1]
    IDMs_by_id=inputs[2]
    types_by_id=inputs[3]
    lens_by_id=inputs[4]
    trips_data=inputs[5]
    is_generated={} # whether the vehicles in the trip file are generated
    for idx in lens_by_id:
       is_generated[idx]=False 
    
    sumo_binary = checkBinary('sumo-gui') # do 
    sim_time=max_t
    config_file = "configs/"+site+"/"+site+"_trial"+str(trial)+".sumocfg" # read the configuration file

    # initialize the data collection dictionary
    all_vehs={"time":[], "veh_id":[], "type":[], "length":[],  "x":[], "y":[], "v":[], "road":[], "lane":[], "lane_pos":[] } # characteristics of vehicles of each step size appended into pd dataframe
    
    print("sim_t", sim_time)

    # start the simulation
    traci.start([sumo_binary, "-c", config_file], verbose=False, label=str(trial))  # Starts the simulation

    step = 0
    lane_lens={lane: traci.lane.getLength(lane) for lane in traci.lane.getIDList()}
    
    while step < int(sim_time/step_size):  # Run the simulation for 3600 seconds (1 hour), which is 36000 steps
        if step*step_size%100==0:
            print(step*step_size)
        
        # vehicle ids
        veh_ids=traci.vehicle.getIDList()


        # dictionary to append new speeds and lanes when calculated
        # it will be updated in SUMO once all the vehicles have been calculated
        
        new_speeds={} # make sure there is no sequence effect
        new_lanes={} # the new lane choice of vehicles
        veh_ids_by_lane={} # vehicle ids in one lane
        
        for veh_id in veh_ids:
            
            
            if is_generated[veh_id]==False: # only set when the vehicles are first step generation
                traci.vehicle.setParameter(veh_id, "lcTimeToLat", "2.0")
                traci.vehicle.setLength(veh_id, lens_by_id[veh_id]) 
                is_generated[veh_id]=True
            
            x_a = traci.vehicle.getLanePosition(veh_id)
            position=traci.vehicle.getPosition(veh_id)
            lane_pos=traci.vehicle.getLanePosition(veh_id)
            v_a=traci.vehicle.getSpeed(veh_id)
            road= traci.vehicle.getRoadID(veh_id)
            lane= traci.vehicle.getLaneIndex(veh_id)

            # append the data in sumo into collected data
            all_vehs["time"].append(step*step_size)
            all_vehs["veh_id"].append(veh_id)
            all_vehs["x"].append(position[0])
            all_vehs["y"].append(position[1])
            all_vehs["v"].append(v_a)
            all_vehs["road"].append(road)
            all_vehs["lane"].append((road)+"_"+str(lane))
            all_vehs["lane_pos"].append(lane_pos)
            all_vehs["type"].append(types_by_id[veh_id])
            all_vehs["length"].append(lens_by_id[veh_id])
            
            
            leader_id=traci.vehicle.getLeader(veh_id) # need to be obtained here
            next_lane=(road)+"_"+str(lane) # initialize as the current lane
            
            
            # find the leader (since it may not have the same lane index as the follower)
            adjust_length= lane_lens[next_lane] - traci.vehicle.getLanePosition(veh_id) #traci.lane.getLength(next_lane)-traci.vehicle.getLanePosition(veh_id) 
            forward_count=0
            most_upstream_dist=None
            
            while leader_id==None and next_lanes_dict[next_lane]!=None and forward_count<3 and adjust_length<500 :
                forward_count=forward_count+1
                
                next_lane=next_lanes_dict[next_lane]
                if not next_lane in veh_ids_by_lane:
                    next_ids=traci.lane.getLastStepVehicleIDs(next_lane)
                    veh_ids_by_lane[next_lane]=next_ids
                else:
                    next_ids=veh_ids_by_lane[next_lane]
                 
                if len(next_ids)==0: # the next lane has 0 vehicles
                    adjust_length=adjust_length+lane_lens[next_lane]#traci.lane.getLength(next_lane) # the lane is skipped, so add up the length of it
                    continue
                
                most_upstream_id = min(next_ids, key=lambda vid: traci.vehicle.getLanePosition(vid))
                most_upstream_dist = traci.vehicle.getLanePosition(most_upstream_id)
                leader_id=[most_upstream_id]
                
            leader_lane=next_lane       

            if leader_id is not None:
                leader_id=leader_id[0]
                leader_len=lens_by_id[leader_id]

            # customize car follow model
            if cf_model=="IDM": # for IDM
                acc=IDM(veh_id, leader_id, leader_lane, adjust_length, x_a, v_a, road, (road)+"_"+str(lane), most_upstream_dist)
                new_speeds[veh_id]=v_a+acc*step_size#max(v_a+acc*step_size, 0) # changed here

            if cf_model=="PT" or cf_model=="Prospect Theory": # for PT
                # implement prospect theory here
                acc=PT(veh_id, leader_id)
                new_speeds[veh_id]=max(v_a+acc*step_size, 0)
            
        for veh_id in veh_ids:
            traci.vehicle.setSpeed(veh_id, new_speeds[veh_id])


        for veh_id in veh_ids:
            # to do: update the lane of the vehicles 

            
        traci.simulationStep()  # Advance one simulation step
        step += 1
    
    traci.close()  # Close the simulation

    return all_vehs

In [ ]:
# pick the site
site_idx=3
sites={1:"I90_94", 2:"I294", 3:"I395" }
site=sites[site_idx]


min_hw=0.1 # the minimum headway between vehicles during vehicle generation. Not sure if this is needed
# the min and max sim time
min_t=0
max_t=1200 # change this for a different simulation time

# simulation step size
step_size=0.1

# Model selection
cf_model_choice=1
cf_models = {1:"IDM", 2:"PT"} # car following model

# number_of trails
num_trials = 1

# warm up time among which data will not be collected
warm_up_t=200

In [ ]:
# the parameter of the scenarios: [av_rate, lv_rate, flow_main, flow_ramp_to_ramp, flow_ramp_to_main, flow_main_to_ramp, cf_model_choice]# 
# feel free to run all of them
all_scenarios=[[0.3, 0.2, 6000, 500, 500, 500, 1]]

In [ ]:
# one at a time
# make sure no ray module before the function
#'''
for scenarios in all_scenarios:
    print(scenarios)
    av_rate, lv_rate, flow_main, flow_ramp_to_ramp, flow_ramp_to_main, flow_main_to_ramp, cf_model_choice = scenarios
    sv_rate=1-av_rate-lv_rate
    cf_model= cf_models[cf_model_choice]
    inputs_all_trial=[]
    od_demand, all_edges, in_edges, out_edges, first_exit, first_exit_start_lane, next_lanes_dict=initialize()
    for trial in range(0,num_trials): # number of trials, we can simulate multiple trials
        print("trial", trial)
        PTs_by_id,  IDMs_by_id, types_by_id, lens_by_id, trips_data= generate_vehicles(trial)
        inputs_all_trial.append([trial, PTs_by_id,  IDMs_by_id, types_by_id, lens_by_id, trips_data])
        
        
    vehs_all_trials=[run_simulation_default(inputs) for inputs in inputs_all_trial]
    
    for trial in range(0,1):
        all_vehs=pd.DataFrame(vehs_all_trials[trial])
        # save the collected vehicle data
        all_vehs.to_csv("trajs_"+site+"_"+str(scenarios)+"_trial"+str(trial)+".csv")
